In [ ]:
import time

APT_LIST_URL = "https://sssb.se/widgets/?callback=a&widgets%5B%5D=objektlista%40lagenheter"
APT_DETAILS_URL_PREFIX = "https://sssb.se/widgets/?widgets%5B%5D=objektinformation%40lagenheter&widgets%5B%5D=objektdokument&widgets%5B%5D=objektintressestatus&widgets%5B%5D=objektintresse&callback=a&refid="
DATA_FILENAME = "sssbppp-data.csv"

# Collecting refIDs:

In [2]:
from bs4 import BeautifulSoup
import requests
import re

page = requests.get(APT_LIST_URL).text
soup = BeautifulSoup(page, 'html.parser')

apt_ids = []
p = re.compile(r'refid=([A-Za-z0-9_.]+)')
for anchor in soup.findAll('a', href=True):
    link = anchor['href']
    ref = p.findall(link)[0]
    apt_ids.append(ref)

## Adding old ids that are expiring:

In [25]:
import os.path
import pandas as pd
import dateutil.parser as timeparser

# Getting the latest scrape
if os.path.exists(DATA_FILENAME):
    df = pd.read_csv(DATA_FILENAME)
    latest_snap = df[df['scrape-time']==df['scrape-time'].max()]
    for idx, apt in latest_snap.iterrows():
        time_string = apt['objektintresse'][0:50]
        apt_close_time = timeparser.parse(time_string, fuzzy=True)
        if time.time() > apt_close_time:
            apt_ids.append(apt["refid"])


# Collecting individual apartment data:

In [6]:
import json

timestamp = time.time()
apts = []
for ref in apt_ids:
    url = APT_DETAILS_URL_PREFIX + ref
    page = str.removeprefix(requests.get(url).text, "a(")
    page = str.removesuffix(page, ");")
    dict = json.loads(page)
    apt = dict["data"]["objektinformation@lagenheter"]

    apt["objektintressestatus"] = dict["html"]['objektintressestatus']

    soup = BeautifulSoup(dict["html"]["objektintresse"], 'html.parser')
    div = soup.findAll('div')[0]
    apt['objektintresse'] = div.text

    apt['scrape-time'] = timestamp

    apts.append(dict["data"]["objektinformation@lagenheter"])

In [7]:
import csv

new_file = not(os.path.isfile(DATA_FILENAME))
with open(DATA_FILENAME, encoding="utf-8", mode="a") as file:
    w = csv.DictWriter(file, apts[0].keys())
    if new_file:
        w.writeheader()

    for apt in apts:
        w.writerow(apt)
